Uncomment the next line to install the `requests` package

In [ ]:
# !pip install requests

Import the required libraries:
* `requests`, for making HTTP GET requests
* `pandas`, for importing CSV files into a dataframe, and viceversa
* `json_normalize`, for parsing the JSON response from the server

In [7]:
import requests 
import pandas as pd 
from pandas.io.json import json_normalize 

We store the Bing API Key in a variable so we can append it to the HTTP requests easily.

Please, update the next cell with your own Bing API key.

In [8]:
apikey=""

Similarly, we will save in variables the coordinates of the commute destination, and the different commute start times that we will use.

In [9]:
destination="38.8548783, -77.0517428"
startTimes=["06:00:00", "06:30:00", "07:00:00", "07:30:00", "08:00:00", "08:30:00", "09:00:00", "09:30:00"]

Now we create the dataframe where we will store the commute times, providing the column names.

In [17]:
commute_pd = pd.DataFrame(columns=["ZCTA5CE10", "06:00:00", "06:30:00", "07:00:00", "07:30:00", "08:00:00", "08:30:00", "09:00:00", "09:30:00"]);

We can now load the contents of the CSV file into a dataframe.

In [18]:
locations=pd.read_csv("prepared_data/zcta_md_dc_va.csv")

And now we get to the main loop of this notebook. The process here is to iterate over the dataframe read from the CSV file (`locations`), reading the ZCTA value and the coordinates for the internal point. For each of these entries, we use for loops to get the combinations of 'commute start time' and 'commute day' (going from March 2nd 2020 (Monday) to March 6th (Friday) for a full week), and for each of these combinations, we build the HTTP request in the variable `url`, send the request, and process the result to obtain the returned commute time. This time we accumulate in the variable `acumCommuteTime` and after iterating over the 5 days of the week, we compute the average (that is, the average of the commute starting at a given time, for the 5 days of the week). This average value (stored in the variable `finalTime`) is pushed into the dataframe of commute times (`commute_pd`). 

After the lengthy process of making all these queries, the dataframe will be fully populated. We show the top and bottom rows for validation.

In [31]:
for i in locations.index:
    rowdata = {"ZCTA5CE10" : str(locations["ZCTA5CE10"][i])}
    for st in startTimes:
        acumCommuteTime=0
        for day in range (2,7):
            url="https://dev.virtualearth.net/REST/v1/Routes/DistanceMatrix?origins=" + str(locations["INTPTLAT10"][i]) + "," + str(locations["INTPTLON10"][i]) + "&destinations=" + destination + "&travelMode=driving&startTime=2020-03-0" + str(day) + "T" + st + "-05:00&key=" + apikey
            results = requests.get(url).json()
            acumCommuteTime += results['resourceSets'][0]['resources'][0]['results'][0]['travelDuration'];
        
        finalTime = acumCommuteTime / 5
        
        rowdata.update({st : finalTime})
    commute_pd = commute_pd.append (rowdata, ignore_index=True)
commute_pd

We now save the dataframe to the CSV file `zcta_commute_times.csv` in the `prepared_data` directory.

In [20]:
commute_pd.to_csv('prepared_data/zcta_commute_times.csv', index=False)